In [22]:
from scipy.signal import savgol_filter
import numpy as np
import pandas as pd
from more_itertools import locate
from datetime import datetime
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from support.pd_support import *

In [23]:
def read_rigid_body_csv(_pth):
    """
    _pth: path to the rigid body file
    """
    df = pd.read_csv(_pth, skiprows=2, header=None)

    # get the start time of the capture
    raw_df = pd.read_csv(_pth)
    cols_list = raw_df.columns     # first row which contains capture start time
    inx = [i for i, x in enumerate(cols_list) if x == "Capture Start Time"]
    st_time = cols_list[inx[0] + 1]
    st_time = datetime.strptime(st_time, "%Y-%m-%d %I.%M.%S.%f %p")  # returns datetime object

    _marker_type = []
    for idx in df.columns:
        _marker_type.append(df[idx][0])

    def find_indices(list_to_check, item_to_find):
        indices = locate(list_to_check, lambda x: x == item_to_find)
        return list(indices)

    # find the indices of the marker types
    _rb_idx = find_indices(_marker_type, "Rigid Body")
    _rb_marker_idx = find_indices(_marker_type, "Rigid Body Marker")
    _marker_idx = find_indices(_marker_type, "Marker")

    _rb_idx.sort()
    _rb_marker_idx.sort()
    _marker_idx.sort()

    _rb_df = df.copy(deep=True)
    _rb_df = df[1:]
    _analysis_type = list(_rb_df.iloc[2].values)

    _rotation_ids = find_indices(_analysis_type, "Rotation")
    _rotation_ids.sort()

    _rb_pos_idx = _rb_idx.copy()

    # remove the rotation indices from the position indices
    [_rb_pos_idx.remove(i) for i in _rotation_ids]

    # create column names for angle
    col_names = []

    # first two columns
    col_names.append("frame")
    col_names.append("time_seconds")

    for i in _rotation_ids:
        _col = _rb_df[i].iloc[3].lower()
        col_names.append("rb_ang_" + _col)

    # create column names for position
    for i in _rb_pos_idx:
        if isinstance(_rb_df[i].iloc[3], str):
            _col = _rb_df[i].iloc[3].lower()
            col_names.append("rb_pos_" + _col)
        else:
            col_names.append("rb_pos_err")


    # rigid body individual marker section

    for i in _rb_marker_idx:
        if isinstance(_rb_df[i].iloc[0], str):
            _col_head = _rb_df[i].iloc[0].lower()
            _col_head = _col_head.split(":")[1].strip()
            _col_head = _col_head.replace("marker", "")
            _m_idx = int(_col_head)
            
            if isinstance(_rb_df[i].iloc[3], str):
                _col = _rb_df[i].iloc[3].lower()
                col_names.append("rb_marker_m" + str(_col_head) + "_" + _col)
            else:
                col_names.append("rb_marker_m" + str(_col_head) + "_mq") # marker quality


    # individual marker section 

    for i in _marker_idx:
        if isinstance(_rb_df[i].iloc[0], str):
            _col_head = _rb_df[i].iloc[0].lower()
            _col_head = _col_head.split(":")[1].strip()
            _col_head = _col_head.replace("marker", "")
            _m_idx = int(_col_head)
            
            if isinstance(_rb_df[i].iloc[3], str):
                _col = _rb_df[i].iloc[3].lower()
                col_names.append("m" + str(_col_head) + "_" + _col)

    _rb_df = _rb_df[4:]
    _rb_df.columns = col_names

    #reset index
    _rb_df = _rb_df.reset_index(drop=True)

    return _rb_df, st_time

In [24]:
_pth = r"C:\Users\CMC\Documents\openposelibs\pose\armbo\recording_programs\test_data\omni_9dof_feb_28_2023\sk27\sk27_00_random_slow.csv"

In [25]:
df, _st_time = read_rigid_body_csv(_pth)
df

c:\Users\CMC\anaconda3\envs\gs\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,frame,time_seconds,rb_ang_x,rb_ang_y,rb_ang_z,rb_ang_w,rb_pos_x,rb_pos_y,rb_pos_z,rb_pos_err,...,m1_z,m2_x,m2_y,m2_z,m3_x,m3_y,m3_z,m4_x,m4_y,m4_z
0,0,0.000000,-0.001507,-0.000727,0.002973,-0.999994,0.039218,0.049546,0.051924,0.000043,...,0.113586,0.003602,0.050147,0.041298,0.051863,0.049464,0.046483,0.055384,0.049272,0.006330
1,1,0.010000,-0.001544,-0.000742,0.003380,-0.999993,0.039217,0.049546,0.051926,0.000046,...,0.113588,0.003605,0.050177,0.041304,0.051864,0.049454,0.046487,0.055379,0.049262,0.006324
2,2,0.020000,-0.001658,-0.000743,0.003239,-0.999993,0.039214,0.049542,0.051923,0.000039,...,0.113590,0.003591,0.050165,0.041296,0.051851,0.049426,0.046472,0.055387,0.049291,0.006333
3,3,0.030000,-0.000246,-0.000477,0.003017,-0.999995,0.039200,0.049592,0.051980,0.000130,...,0.113824,0.003602,0.050169,0.041298,0.051858,0.049419,0.046472,0.055369,0.049249,0.006325
4,4,0.040000,-0.001543,-0.000796,0.003336,-0.999993,0.039208,0.049542,0.051928,0.000049,...,0.113596,0.003595,0.050170,0.041304,0.051845,0.049434,0.046466,0.055367,0.049271,0.006344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32572,32572,325.72,-0.001504,-0.147252,0.001158,-0.989097,-0.107417,0.05015,0.060467,0.000175,...,0.117661,-0.144485,0.050608,0.060545,-0.097056,0.050108,0.051555,-0.105171,0.049965,0.012108
32573,32573,325.73,0.000245,-0.147486,0.003015,-0.98906,-0.10707,0.050176,0.05782,0.000192,...,0.115002,-0.144193,0.050752,0.057884,-0.096791,0.050132,0.048928,-0.104784,0.049748,0.009468
32574,32574,325.74,-0.000866,-0.147117,0.001326,-0.989118,-0.106684,0.050186,0.055181,0.000233,...,0.112439,-0.143766,0.05065,0.055241,-0.096336,0.049984,0.046194,-0.104417,0.05003,0.006849
32575,32575,325.75,-0.001216,-0.147244,0.002086,-0.989097,-0.106257,0.050106,0.052487,0.000215,...,0.109735,-0.143321,0.050628,0.052574,-0.095887,0.050051,0.043506,-0.104021,0.049868,0.00413


In [26]:
"""
_pth: path to the rigid body file
"""
df = pd.read_csv(_pth, skiprows=2, header=None, dtype=str)

# get the start time of the capture
raw_df = pd.read_csv(_pth, dtype=str)
cols_list = raw_df.columns     # first row which contains capture start time
inx = [i for i, x in enumerate(cols_list) if x == "Capture Start Time"]
st_time = cols_list[inx[0] + 1]
st_time = datetime.strptime(st_time, "%Y-%m-%d %I.%M.%S.%f %p")  # returns datetime object

_marker_type = []
for idx in df.columns:
    _marker_type.append(df[idx][0])

def find_indices(list_to_check, item_to_find):
    indices = locate(list_to_check, lambda x: x == item_to_find)
    return list(indices)

# find the indices of the marker types
_rb_idx = find_indices(_marker_type, "Rigid Body")
_rb_marker_idx = find_indices(_marker_type, "Rigid Body Marker")
_marker_idx = find_indices(_marker_type, "Marker")

_rb_idx.sort()
_rb_marker_idx.sort()
_marker_idx.sort()

_rb_df = df.copy(deep=True)
_rb_df = df[1:]
_analysis_type = list(_rb_df.iloc[2].values)

_rotation_ids = find_indices(_analysis_type, "Rotation")
_rotation_ids.sort()

_rb_pos_idx = _rb_idx.copy()

# remove the rotation indices from the position indices
[_rb_pos_idx.remove(i) for i in _rotation_ids]

# create column names for angle
col_names = []

# first two columns
col_names.append("frame")
col_names.append("seconds")

for i in _rotation_ids:
    _col = _rb_df[i].iloc[3].lower()
    col_names.append("rb_ang_" + _col)

# create column names for position
for i in _rb_pos_idx:
    if isinstance(_rb_df[i].iloc[3], str):
        _col = _rb_df[i].iloc[3].lower()
        col_names.append("rb_pos_" + _col)
    else:
        col_names.append("rb_pos_err")


# rigid body individual marker section

for i in _rb_marker_idx:
    if isinstance(_rb_df[i].iloc[0], str):
        _col_head = _rb_df[i].iloc[0].lower()
        _col_head = _col_head.split(":")[1].strip()
        _col_head = _col_head.replace("marker", "")
        _m_idx = int(_col_head)
        
        if isinstance(_rb_df[i].iloc[3], str):
            _col = _rb_df[i].iloc[3].lower()
            col_names.append("rb_marker_m" + str(_col_head) + "_" + _col)
        else:
            col_names.append("rb_marker_m" + str(_col_head) + "_mq") # marker quality


# individual marker section 

for i in _marker_idx:
    if isinstance(_rb_df[i].iloc[0], str):
        _col_head = _rb_df[i].iloc[0].lower()
        _col_head = _col_head.split(":")[1].strip()
        _col_head = _col_head.replace("marker", "")
        _m_idx = int(_col_head)
        
        if isinstance(_rb_df[i].iloc[3], str):
            _col = _rb_df[i].iloc[3].lower()
            col_names.append("m" + str(_col_head) + "_" + _col)

_rb_df = _rb_df[4:]
_rb_df.columns = col_names

#reset index
_rb_df = _rb_df.reset_index(drop=True)
_rb_df = _rb_df.apply(pd.to_numeric, errors='ignore')



In [30]:
_rb_df

,frame,time_seconds,rb_ang_x,rb_ang_y,rb_ang_z,rb_ang_w,rb_pos_x,rb_pos_y,rb_pos_z,rb_pos_err,...,m1_z,m2_x,m2_y,m2_z,m3_x,m3_y,m3_z,m4_x,m4_y,m4_z
0,0,0.00,-0.001507,-0.000727,0.002973,-0.999994,0.039218,0.049546,0.051924,0.000043,...,0.113586,0.003602,0.050147,0.041298,0.051863,0.049464,0.046483,0.055384,0.049272,0.006330
1,1,0.01,-0.001544,-0.000742,0.003380,-0.999993,0.039217,0.049546,0.051926,0.000046,...,0.113588,0.003605,0.050177,0.041304,0.051864,0.049454,0.046487,0.055379,0.049262,0.006324
2,2,0.02,-0.001658,-0.000743,0.003239,-0.999993,0.039214,0.049542,0.051923,0.000039,...,0.113590,0.003591,0.050165,0.041296,0.051851,0.049426,0.046472,0.055387,0.049291,0.006333
3,3,0.03,-0.000246,-0.000477,0.003017,-0.999995,0.039200,0.049592,0.051980,0.000130,...,0.113824,0.003602,0.050169,0.041298,0.051858,0.049419,0.046472,0.055369,0.049249,0.006325
4,4,0.04,-0.001543,-0.000796,0.003336,-0.999993,0.039208,0.049542,0.051928,0.000049,...,0.113596,0.003595,0.050170,0.041304,0.051845,0.049434,0.046466,0.055367,0.049271,0.006344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32572,32572,325.72,-0.001504,-0.147252,0.001158,-0.989097,-0.107417,0.050150,0.060467,0.000175,...,0.117661,-0.144485,0.050608,0.060545,-0.097056,0.050108,0.051555,-0.105171,0.049965,0.012108
32573,32573,325.73,0.000245,-0.147486,0.003015,-0.989060,-0.107070,0.050176,0.057820,0.000192,...,0.115002,-0.144193,0.050752,0.057884,-0.096791,0.050132,0.048928,-0.104784,0.049748,0.009468
32574,32574,325.74,-0.000866,-0.147117,0.001326,-0.989118,-0.106684,0.050186,0.055181,0.000233,...,0.112439,-0.143766,0.050650,0.055241,-0.096336,0.049984,0.046194,-0.104417,0.050030,0.006849
32575,32575,325.75,-0.001216,-0.147244,0.002086,-0.989097,-0.106257,0.050106,0.052487,0.000215,...,0.109735,-0.143321,0.050628,0.052574,-0.095887,0.050051,0.043506,-0.104021,0.049868,0.004130


In [28]:
_rb_df

,frame,time_seconds,rb_ang_x,rb_ang_y,rb_ang_z,rb_ang_w,rb_pos_x,rb_pos_y,rb_pos_z,rb_pos_err,...,m1_z,m2_x,m2_y,m2_z,m3_x,m3_y,m3_z,m4_x,m4_y,m4_z
0,0,0.00,-0.001507,-0.000727,0.002973,-0.999994,0.039218,0.049546,0.051924,0.000043,...,0.113586,0.003602,0.050147,0.041298,0.051863,0.049464,0.046483,0.055384,0.049272,0.006330
1,1,0.01,-0.001544,-0.000742,0.003380,-0.999993,0.039217,0.049546,0.051926,0.000046,...,0.113588,0.003605,0.050177,0.041304,0.051864,0.049454,0.046487,0.055379,0.049262,0.006324
2,2,0.02,-0.001658,-0.000743,0.003239,-0.999993,0.039214,0.049542,0.051923,0.000039,...,0.113590,0.003591,0.050165,0.041296,0.051851,0.049426,0.046472,0.055387,0.049291,0.006333
3,3,0.03,-0.000246,-0.000477,0.003017,-0.999995,0.039200,0.049592,0.051980,0.000130,...,0.113824,0.003602,0.050169,0.041298,0.051858,0.049419,0.046472,0.055369,0.049249,0.006325
4,4,0.04,-0.001543,-0.000796,0.003336,-0.999993,0.039208,0.049542,0.051928,0.000049,...,0.113596,0.003595,0.050170,0.041304,0.051845,0.049434,0.046466,0.055367,0.049271,0.006344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32572,32572,325.72,-0.001504,-0.147252,0.001158,-0.989097,-0.107417,0.050150,0.060467,0.000175,...,0.117661,-0.144485,0.050608,0.060545,-0.097056,0.050108,0.051555,-0.105171,0.049965,0.012108
32573,32573,325.73,0.000245,-0.147486,0.003015,-0.989060,-0.107070,0.050176,0.057820,0.000192,...,0.115002,-0.144193,0.050752,0.057884,-0.096791,0.050132,0.048928,-0.104784,0.049748,0.009468
32574,32574,325.74,-0.000866,-0.147117,0.001326,-0.989118,-0.106684,0.050186,0.055181,0.000233,...,0.112439,-0.143766,0.050650,0.055241,-0.096336,0.049984,0.046194,-0.104417,0.050030,0.006849
32575,32575,325.75,-0.001216,-0.147244,0.002086,-0.989097,-0.106257,0.050106,0.052487,0.000215,...,0.109735,-0.143321,0.050628,0.052574,-0.095887,0.050051,0.043506,-0.104021,0.049868,0.004130


In [29]:
_rb_df.rb_ang_w

0       -0.999994
1       -0.999993
2       -0.999993
3       -0.999995
4       -0.999993
           ...   
32572   -0.989097
32573   -0.989060
32574   -0.989118
32575   -0.989097
32576   -0.989176
Name: rb_ang_w, Length: 32577, dtype: float64